In [ ]:
import os
import sys

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from ase.build import make_supercell
from ase import Atoms
from ase.optimize import BFGS, LBFGS
from HybridFF import HybridFF
from Utilities import show_results, write_xyz, validate
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM

In [ ]:
DATA = np.load('../../../data/test_sets/DATA_CSP_ranking.npy', allow_pickle=True).item()

In [ ]:
hybrid_ff = HybridFF(debug=True, pol0=False)

In [ ]:
SYSTEM = 'XXIII'
RESULTS = {}
RESULTS[SYSTEM] = {}
MIN_STRUCTURES = {}
THERMAL_CELLS = np.load(f'data/THERMAL_CELLS_{SYSTEM}.npy', allow_pickle=True).item()
MIN_STRUCTURES = np.load(f'data/MINIMIZED_THERMAL_STRUCTURES_{SYSTEM}.npy', allow_pickle=True).item()
RESULTS = np.load(f'data/THERMAL_RESULTS_{SYSTEM}.npy', allow_pickle=True).item()

In [ ]:
for poly_key in DATA[SYSTEM]:
    if poly_key not in THERMAL_CELLS:
        print(poly_key)

In [ ]:
for poly_key in THERMAL_CELLS:
    if poly_key not in MIN_STRUCTURES:
        RESULTS[SYSTEM][poly_key] = {}
        MIN_STRUCTURES[poly_key] = {}
        print(poly_key, DATA[SYSTEM][poly_key]['PBE+MBD'])
        mol_size = len(DATA[SYSTEM][poly_key]['monomer_symbols'])
        elements_uc = DATA[SYSTEM][poly_key]['uc_symbols']
        coords_uc = THERMAL_CELLS[poly_key]['coords']
        cells = THERMAL_CELLS[poly_key]['cells']
        atoms = hybrid_ff.initialize(elements_uc, coords_uc, cells, mol_size)    
        optimizer = LBFGS(atoms) # BFGS
        optimizer.run(fmax=1 / EV_TO_KJ)
        RESULTS[SYSTEM][poly_key]['total'] = atoms.get_potential_energy() * EV_TO_KJ / hybrid_ff._n_molecules_uc
        MIN_STRUCTURES[poly_key]['coordinates'] = atoms.get_positions()[None]    
        MIN_STRUCTURES[poly_key]['cell'] = cells
        MIN_STRUCTURES[poly_key]['uc_symbols'] = elements_uc
        np.save(f'data/MINIMIZED_THERMAL_STRUCTURES_{SYSTEM}.npy', MIN_STRUCTURES)
        np.save(f'data/THERMAL_RESULTS_{SYSTEM}.npy', RESULTS)